In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from util.db_util import DatabaseUtility


In [8]:
import sys
print(sys.executable)


/usr/local/opt/python@3.10/bin/python3.10


In [7]:
import pandas as pd
from util.db_util import DatabaseUtility

# Initialize database connection
db_util = DatabaseUtility()

# Execute query
db_util.cursor.execute("""
    SELECT * FROM wr_model_data;
""")

# Fetch all rows
rows = db_util.cursor.fetchall()

# Get column names from cursor description
columns = [desc[0] for desc in db_util.cursor.description]

# Create a DataFrame
df = pd.DataFrame(rows, columns=columns)

# Close the database connection
db_util.conn.close()

# Display the first few rows of the DataFrame
print(df.head())


InterfaceError: 2002: Can't connect to local MySQL server through socket '%-.100s' (%s) (Warning: not all arguments converted during string formatting)

In [ ]:
# Database connection details (Update with actual credentials)
db_config = {
    "host": "your_host",
    "user": "your_user",
    "password": "your_password",
    "database": "your_database"
}

# Load data from MySQL
conn = mysql.connector.connect(**db_config)
query = "SELECT * FROM wr_model_data;"
df = pd.read_sql(query, conn)
conn.close()

# Preprocessing: Drop non-numeric columns
df = df.drop(columns=["player_id", "name", "position"], errors='ignore')

# Handle missing values by filling with 0
df = df.fillna(0)

# Define features (X) and target variables (y)
X = df.drop(columns=["avg_fppg_nfl", "total_fantasy_points_nfl"])
y_avg_fppg = df["avg_fppg_nfl"]
y_total_fantasy = df["total_fantasy_points_nfl"]

# Train-test split (80% train, 20% test)
X_train, X_test, y_train_fppg, y_test_fppg = train_test_split(X, y_avg_fppg, test_size=0.2, random_state=42)
X_train, X_test, y_train_fantasy, y_test_fantasy = train_test_split(X, y_total_fantasy, test_size=0.2, random_state=42)

# Train XGBoost model for avg_fppg_nfl
xgb_model_fppg = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
xgb_model_fppg.fit(X_train, y_train_fppg)

# Train XGBoost model for total_fantasy_points_nfl
xgb_model_fantasy = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)
xgb_model_fantasy.fit(X_train, y_train_fantasy)

# Make predictions
y_pred_fppg = xgb_model_fppg.predict(X_test)
y_pred_fantasy = xgb_model_fantasy.predict(X_test)

# Evaluate model performance
mae_fppg = mean_absolute_error(y_test_fppg, y_pred_fppg)
rmse_fppg = np.sqrt(mean_squared_error(y_test_fppg, y_pred_fppg))

mae_fantasy = mean_absolute_error(y_test_fantasy, y_pred_fantasy)
rmse_fantasy = np.sqrt(mean_squared_error(y_test_fantasy, y_pred_fantasy))

# Display results
print(f"Model Performance for avg_fppg_nfl: MAE={mae_fppg:.2f}, RMSE={rmse_fppg:.2f}")
print(f"Model Performance for total_fantasy_points_nfl: MAE={mae_fantasy:.2f}, RMSE={rmse_fantasy:.2f}")

# Example prediction for a new player (First row in dataset)
new_player = X.iloc[0].values.reshape(1, -1)
predicted_fppg = xgb_model_fppg.predict(new_player)[0]
predicted_fantasy = xgb_model_fantasy.predict(new_player)[0]

print(f"Predicted NFL FPPG: {predicted_fppg:.2f}")
print(f"Predicted Total Fantasy Points: {predicted_fantasy:.2f}")